In [19]:
# UT-TOR-DATA-PT-01-2020-U-C Team project #1
# (c) Boris Smirnov

## The problem


CSV files from Elections Canada web site are very inconsistent in regard of use of the political parties names and the provinces names:

* The candidates files (`table_tableau11.csv`, `table_tableau12.csv`) have party names as part of winning candidates column - party name is just *appended* to the end of every line

* The file with number of valid votes by political affiliation (`table_tableau08.csv`) have parties names that differ from `table_tableau11.csv`. Provinces names are abbreaviated.

* Geodata files use traditional full provinces names as pairs English name/French name (when there is a French name)

* Files generated by other team members use only winning parties names in a short form ('liberal', 'conservative', 'NDP', etc.)

* Files generated by other team members refer to provinces either by full English name or by numeric id (which is good)

In general, files provided by Elections Canada are better for *looking at*, then for computer processing.


## The Goal


* Create dictionaries that combine all variants of parties and provinces names, that can be used as **translation tables** for merging data from different sources.

* Same dictionaries also provide short **Ids** that can be used for indexing data frames.


## Data Sources


* Elections Canada [42nd General Election: Official Voting Results (raw data)](https://www.elections.ca/content.aspx?section=res&dir=rep/off/42gedata&document=summary&lang=e)
    - Table 8: Number of valid votes by political affiliation
    - Table 11: Voting results by electoral district
    - Table 12: List of candidates by electoral district and individual results


* Election atlas project: [Party Abbreviations](http://www.election-atlas.ca/about.php)
    - acronyms, used for id columns in data frames


* Statistics Canada: [Abbreviations and codes for provinces and territories, 2016 Census](https://www12.statcan.gc.ca/census-recensement/2016/ref/dict/geo038-eng.cfm)
    - provinces.tab - copy-pasted the table into UTF-8 text file, fixed the header, interpreted as tab-separated text


## Resulting dictionaries


* **all_parties_2015.csv** - complete list of the parties that participated in 2015 elections
* **winning_parties_2015.csv** - only the list of winning parties of 2015 elections
* **provinces_ids.csv** - all standard ways to identify provinces and territories


## Columns specifications in political parties dictionaries


* **Id** - unique 2-4 charcters Id that can be use as an index or a key
* **Long Name (en)**, **Long Name (fr)** - full name of a party as in parties related data files
* **Short Name (en)**, **Short Name (fr)** - short name (not always: "NDP-New Democratic Party" isn't short at all) of a party as used in data files related to candidates
* **Candidate Suffix** - a suffix appended to a candidate name in candidates related data files (can be used to isolate candidate name by substring matching of suffixes from this dictionary)
* **Custom Name** - names of winning parties used in our project data files/frames

In [20]:
# Depedences and initialization
import pandas as pd

# Input files:
votes_party_fname = 'table_tableau08.csv'
voting_results_fname = 'table_tableau11.csv'
candidates_fname = 'table_tableau12.csv'
prov_tab_fname = 'provinces.tab'

# Output files:
parties_fname = 'all_parties_2015.csv'
winners_fname = 'winning_parties_2015.csv'
provinces_fname = 'provinces_ids.csv'

#### Part 1: Making dictionaries of political parties

In [21]:
# Reading the list of all the parties that participated in the 2015 elections
votes_party_df = pd.read_csv(votes_party_fname)

In [22]:
# Starting to build a data frame with output dictionary
parties_df = pd.DataFrame(votes_party_df['Political affiliation/Appartenance politique'])

# Here we manually match the parties list with their short Ids
parties_df['Id'] = pd.Series([
    'ATN',   'AA',  'BQ', 'CAN', 'CAP', # 0 - 4
    'CHP',  'COM', 'CPC', 'DAP',  'FD', # 5 - 9
    'GRN',  'LIB', 'LTN', 'MAR',  'ML', # 10 - 14
    'NDP', 'PACT', 'PIR',  'PC', 'RHI', # 15 - 19
    'SEN',  'BDG', 'UNI', 'IND', 'NOA'  # 20 - 24
])

# In the original file all parties names were in the form of their English and French variants separated by '/'
# Not surprisingly, it doesn't mean, that both parts should be different:
# 'Animal Alliance Environment Voters Party of Canada' and 'Bloc Québécois' are represented 'As Is' in both languages
# there is always a balance...
parties_df['Long Name (en)'] = parties_df.iloc[:, 0].map(lambda s: s.split('/')[0])
parties_df['Long Name (fr)'] = parties_df.iloc[:, 0].map(lambda s: s.split('/')[1])
parties_df.drop(columns=['Political affiliation/Appartenance politique'], inplace=True)

parties_df

,Id,Long Name (en),Long Name (fr)
0,ATN,Alliance of the North,Alliance du Nord
1,AA,Animal Alliance Environment Voters Party of Ca...,Animal Alliance Environment Voters Party of Ca...
2,BQ,Bloc Québécois,Bloc Québécois
3,CAN,Canada Party,Parti Canada
4,CAP,Canadian Action Party,Parti action canadienne
5,CHP,Christian Heritage Party of Canada,Parti de l'Héritage Chrétien du Canada
6,COM,Communist Party of Canada,Parti communiste du Canada
7,CPC,Conservative Party of Canada,Parti conservateur du Canada
8,DAP,Democratic Advancement Party of Canada,Parti pour l'Avancement de la Démocratie au Ca...
9,FD,Forces et Démocratie,Forces et Démocratie


In [23]:
# Reading the list of all participating candidates.
# Note, that their names are augmented with party affiliation (in both languages separated by '/')
# But it doesn't mean at all, that those parties names match to the names from the dataset above
# And it also doesn't mean, that using '/' as a separator is a reliable approach, e.g.:
#                              Animal Alliance/Environment Voters/Animal Alliance/Environment Voters
candidates_df = pd.read_csv(candidates_fname)

In [24]:
candidates_df['Short Name (fr)'] = candidates_df['Candidate/Candidat'].map(lambda s: s.rsplit('/')[-1])
short_names_df = pd.DataFrame(candidates_df['Short Name (fr)'].unique())
short_names_df.columns = ['Short Name (fr)']
# manual work here
short_names_df['Id'] = pd.Series([
     'LIB', 'NOA', 'NDP', 'CPC', 'GRN',
      'FD', 'COM', 'CHP', 'IND',  'ML',
     'RHI', 'LTN',  'BQ', 'ATN', 'UNI',
      'AA', 'CAP',  'PC', 'MAR', 'PIR',
    'PACT', 'SEN', 'BDG', 'CAN', 'DAP'
])
short_names_df.iloc[15, 0] = 'Animal Alliance/Environment Voters' # ad hoc case
short_names_df

,Short Name (fr),Id
0,Libéral,LIB
1,Aucune appartenance,NOA
2,NPD-Nouveau Parti démocratique,NDP
3,Conservateur,CPC
4,Parti Vert,GRN
5,Forces et Démocratie - Allier les forces de no...,FD
6,Communiste,COM
7,Parti de l'Héritage Chrétien,CHP
8,Indépendant,IND
9,Marxiste-Léniniste,ML


In [25]:
# Now, having the dictionary of the French parties names, we can manually find their English counterparts
parties_list = []
first_candidates_list = []

for i, row in candidates_df.iterrows():
    party_fr = row['Candidate/Candidat'].rsplit('/')[-1]
    if not party_fr in parties_list:
        parties_list.append(party_fr)
        first_candidates_list.append(row['Candidate/Candidat'])

print(first_candidates_list)

['Ken McDonald Liberal/Libéral', 'Scott Andrews ** No Affiliation/Aucune appartenance', 'Jeannie Baldwin NDP-New Democratic Party/NPD-Nouveau Parti démocratique', 'Lorraine E. Barnett Conservative/Conservateur', 'Krista Byrne-Puumala Green Party/Parti Vert', 'Jennifer McCreath Forces et Démocratie - Allier les forces de nos régions/Forces et Démocratie - Allier les forces de nos régions', 'Sean Burton Communist/Communiste', "Christene Squires Christian Heritage Party/Parti de l'Héritage Chrétien", 'Alexander J. MacKenzie Independent/Indépendant', 'Allan Bezanson Marxist-Leninist/Marxiste-Léniniste', 'Megan Brown-Hodges Rhinoceros/Rhinocéros', 'Wayne James Hiscock Libertarian/Libertarien', 'Luc Ferland Bloc Québécois/Bloc Québécois', 'François Belanger ATN/ADN', 'Bob Kesic United Party/Parti Uni', 'Kyle Bowles Animal Alliance/Environment Voters/Animal Alliance/Environment Voters', 'Jeff Sakula CAP/PAC', 'Rob Wolvin PC Party/Parti PC', 'Kornelis Klevering Radical Marijuana/Radical Mariju

In [26]:
# Copy-paste English names from the above
short_names_df['Short Name (en)'] = pd.Series([
    'Liberal', 'No Affiliation', 'NDP-New Democratic Party', 'Conservative', 'Green Party',
    'Forces et Démocratie - Allier les forces de nos régions', 'Communist', 'Christian Heritage Party', 'Independent',
    'Marxist-Leninist',
     'Rhinoceros', 'Libertarian',  'Bloc Québécois', 'ATN', 'United Party',
      'Animal Alliance/Environment Voters', 'CAP',  'PC Party', 'Marijuana', 'Pirate',
    'PACT', 'Seniors Party', 'The Bridge', 'Canada Party', 'Democratic Advancement'
])

# And finally, reconstructing the suffixes
short_names_df['Candidate Suffix'] = ' ' + short_names_df['Short Name (en)'] + '/' + short_names_df['Short Name (fr)']
short_names_df

,Short Name (fr),Id,Short Name (en),Candidate Suffix
0,Libéral,LIB,Liberal,Liberal/Libéral
1,Aucune appartenance,NOA,No Affiliation,No Affiliation/Aucune appartenance
2,NPD-Nouveau Parti démocratique,NDP,NDP-New Democratic Party,NDP-New Democratic Party/NPD-Nouveau Parti dé...
3,Conservateur,CPC,Conservative,Conservative/Conservateur
4,Parti Vert,GRN,Green Party,Green Party/Parti Vert
5,Forces et Démocratie - Allier les forces de no...,FD,Forces et Démocratie - Allier les forces de no...,Forces et Démocratie - Allier les forces de n...
6,Communiste,COM,Communist,Communist/Communiste
7,Parti de l'Héritage Chrétien,CHP,Christian Heritage Party,Christian Heritage Party/Parti de l'Héritage ...
8,Indépendant,IND,Independent,Independent/Indépendant
9,Marxiste-Léniniste,ML,Marxist-Leninist,Marxist-Leninist/Marxiste-Léniniste


In [27]:
# Merging the result of parties names reconstruction from candidates dataset to our universal dictionary
parties_df = pd.merge(parties_df, short_names_df, how='left', on='Id')

In [28]:
# Adding to the data frame custom winning parties names as used by other team members
parties_df['Custom Name'] = ''
parties_df.loc[parties_df['Id'] == 'BQ', 'Custom Name'] = 'Bloc Québécois'
parties_df.loc[parties_df['Id'] == 'CPC', 'Custom Name'] = 'Conservative'
parties_df.loc[parties_df['Id'] == 'GRN', 'Custom Name'] = 'Green Party'
parties_df.loc[parties_df['Id'] == 'LIB', 'Custom Name'] = 'Liberal'
parties_df.loc[parties_df['Id'] == 'NDP', 'Custom Name'] = 'NDP'

In [29]:
# One dictionary done, and it demonstrates what a mess is Elections Canada data. Hope they did the votes right...
parties_df.to_csv(parties_fname, index=False, encoding='utf-8')
parties_df

,Id,Long Name (en),Long Name (fr),Short Name (fr),Short Name (en),Candidate Suffix,Custom Name
0,ATN,Alliance of the North,Alliance du Nord,ADN,ATN,ATN/ADN,
1,AA,Animal Alliance Environment Voters Party of Ca...,Animal Alliance Environment Voters Party of Ca...,Animal Alliance/Environment Voters,Animal Alliance/Environment Voters,Animal Alliance/Environment Voters/Animal All...,
2,BQ,Bloc Québécois,Bloc Québécois,Bloc Québécois,Bloc Québécois,Bloc Québécois/Bloc Québécois,Bloc Québécois
3,CAN,Canada Party,Parti Canada,Parti Canada,Canada Party,Canada Party/Parti Canada,
4,CAP,Canadian Action Party,Parti action canadienne,PAC,CAP,CAP/PAC,
5,CHP,Christian Heritage Party of Canada,Parti de l'Héritage Chrétien du Canada,Parti de l'Héritage Chrétien,Christian Heritage Party,Christian Heritage Party/Parti de l'Héritage ...,
6,COM,Communist Party of Canada,Parti communiste du Canada,Communiste,Communist,Communist/Communiste,
7,CPC,Conservative Party of Canada,Parti conservateur du Canada,Conservateur,Conservative,Conservative/Conservateur,Conservative
8,DAP,Democratic Advancement Party of Canada,Parti pour l'Avancement de la Démocratie au Ca...,Avancement de la Démocratie,Democratic Advancement,Democratic Advancement/Avancement de la Démoc...,
9,FD,Forces et Démocratie,Forces et Démocratie,Forces et Démocratie - Allier les forces de no...,Forces et Démocratie - Allier les forces de no...,Forces et Démocratie - Allier les forces de n...,


In [30]:
# Reading the list of winning candidates (and their party affiliation)
voting_results_df = pd.read_csv(voting_results_fname)

In [31]:
# To speed things up let's use simple (but not exactly reliable) approach of reverse-slash-splitting
# Could have just used 'Custom Name' filter on parties_df , cause we all know who the winners are... still, let's confirm it
voting_results_df['Short Name (fr)'] = voting_results_df['Elected Candidate/Candidat élu'].map(lambda s: s.rsplit('/')[-1])
winning_parties_s = voting_results_df['Short Name (fr)'].unique()
winning_parties_s

array(['Libéral', 'NPD-Nouveau Parti démocratique', 'Conservateur',
       'Bloc Québécois', 'Parti Vert'], dtype=object)

In [32]:
# Finally, the dictionary of winning parties names
winning_parties_df = parties_df.loc[parties_df['Short Name (fr)'].isin(winning_parties_s)]
winning_parties_df.to_csv(winners_fname, index=False, encoding='utf-8')
winning_parties_df

,Id,Long Name (en),Long Name (fr),Short Name (fr),Short Name (en),Candidate Suffix,Custom Name
2,BQ,Bloc Québécois,Bloc Québécois,Bloc Québécois,Bloc Québécois,Bloc Québécois/Bloc Québécois,Bloc Québécois
7,CPC,Conservative Party of Canada,Parti conservateur du Canada,Conservateur,Conservative,Conservative/Conservateur,Conservative
10,GRN,Green Party of Canada,Le Parti Vert du Canada,Parti Vert,Green Party,Green Party/Parti Vert,Green Party
11,LIB,Liberal Party of Canada,Parti libéral du Canada,Libéral,Liberal,Liberal/Libéral,Liberal
15,NDP,New Democratic Party,Nouveau Parti démocratique,NPD-Nouveau Parti démocratique,NDP-New Democratic Party,NDP-New Democratic Party/NPD-Nouveau Parti dé...,NDP


#### Part 2: Making dictionaries of the provinces and the territories

It's a copy of the table [here](https://www12.statcan.gc.ca/census-recensement/2016/ref/dict/geo038-eng.cfm).

Pre-Processing:
* Copy-pasted web-page table as an UTF-8 text file provinces.tab
* Edited column headers line (they were long and splitted to several lines)

In [33]:
provinces_df = pd.read_csv(prov_tab_fname, sep='\t')
provinces_df.rename(columns={
    'Province/Territory': 'Province Name (en)',
    'Internationally approved alpha code (Source: Canada Post)': 'Alpha code',
    'Standard geographical classification (SGC) code': 'Province Id'
}, inplace=True)

# Split abbreviations to separate columns for English and French
provinces_df['Abbreviation (en)'] = provinces_df.iloc[:, 1].map(lambda s: s.split('/')[0])
provinces_df['Abbreviation (fr)'] = provinces_df.iloc[:, 1].map(lambda s: s.split('/')[1])

# Only French provinces names are missing... I'll add them manually
# In Elections Canada data files half of the provinces miss French names and only represented as a single English name
# but those that do have French name are combined with English part in the form of 'Province Name (en)'/'Province Name (fr)'
# as in table_tableau11.csv
provinces_df['Province Name (fr)'] = [
    'Terre-Neuve-et-Labrador',
    'Île-du-Prince-Édouard',
    'Nouvelle-Écosse',
    'Nouveau-Brunswick',
    'Québec',
    '', # Ontario
    '', # Manitoba
    '', # Saskatchewan
    '', # Alberta
    'Colombie-Britannique',
    '', # Yukon
    'Territoires du Nord-Ouest',
    '' # Nunavut
]

# Rearrange the columns
provinces_df = provinces_df.iloc[:, [3, 0, 7, 5, 6, 2, 4]]

# Final dictionary
provinces_df.to_csv(provinces_fname, index=False, encoding='utf-8')
provinces_df

,Province Id,Province Name (en),Province Name (fr),Abbreviation (en),Abbreviation (fr),Alpha code,Region name
0,10,Newfoundland and Labrador,Terre-Neuve-et-Labrador,N.L.,T.-N.-L.,NL,Atlantic
1,11,Prince Edward Island,Île-du-Prince-Édouard,P.E.I.,Î.-P.-É.,PE,Atlantic
2,12,Nova Scotia,Nouvelle-Écosse,N.S.,N.-É.,NS,Atlantic
3,13,New Brunswick,Nouveau-Brunswick,N.B.,N.-B.,NB,Atlantic
4,24,Quebec,Québec,Que.,Qc,QC,Quebec
5,35,Ontario,,Ont.,Ont.,ON,Ontario
6,46,Manitoba,,Man.,Man.,MB,Prairies
7,47,Saskatchewan,,Sask.,Sask.,SK,Prairies
8,48,Alberta,,Alta.,Alb.,AB,Prairies
9,59,British Columbia,Colombie-Britannique,B.C.,C.-B.,BC,British Columbia
